In [17]:
import pandas as pd
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from sklearn.utils.class_weight import compute_class_weight

In [18]:
metadata = pd.read_csv('data/Chest_xray_Corona_Metadata.csv').drop('Unnamed: 0', axis=1)

In [19]:
metadata.head()

,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [20]:
combi = list(set(zip(metadata.Label.tolist(), metadata.Label_1_Virus_category.tolist())))
combi

[('Normal', nan),
 ('Pnemonia', 'Virus'),
 ('Pnemonia', 'bacteria'),
 ('Pnemonia', 'Stress-Smoking')]

In [21]:
fig, axs = plt.subplots(2, 2, figsize=(8,8))
axs = axs.reshape(-1)
i = 0
for c in combi:
    label, cat = c[0], c[1]
    filename = (
        metadata[(metadata['Label'] == label) & ((metadata['Label_1_Virus_category'] == cat) | (metadata['Label_1_Virus_category']
        .isnull()))]
        .iloc[0]
        .X_ray_image_name
    )
    img = Image.open(f'data/train/{filename}')
    img.show()
    axs[i].imshow(img)
    axs[i].title.set_text(f'{c[0]} - {c[1]}')
    i += 1

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'data/train/IM-0128-0001.jpeg'

In [ ]:
# move images

image_root_path = 'data/'
folder_map = {'Normal': 'class_0/', 'Pnemonia': 'class_1/'}
for row in metadata.itertuples():
    filename = row[1]
    folder = folder_map[row[2]]
    train_test = row[3].lower() +'/'
    current_location = f'{image_root_path}{train_test}{filename}'
    new_location = f'{image_root_path}{train_test}{folder}{filename}'
    os.rename(current_location, new_location)

In [22]:
train_data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=0.2,
    height_shift_range=0.1,
    width_shift_range=0.1,
    zoom_range=0.05,
    brightness_range=(-5, 5),
    validation_split=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [23]:
# Calculate class weights to balance data

unique_classes = metadata.Label.unique()
all_rows = metadata.Label.to_numpy()
weights = compute_class_weight('balanced', classes=unique_classes, y=all_rows)

In [28]:
train_images = train_data_gen.flow_from_directory(
    image_root_path + 'train',
    class_mode='binary',
    subset='training'
)

validate_images = train_data_gen.flow_from_directory(
    image_root_path + 'train',
    class_mode='binary',
    subset='validation'
)

test_images = train_data_gen.flow_from_directory(
    image_root_path + 'test',
    class_mode='binary'
)

Found 4230 images belonging to 2 classes.
Found 1056 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Found 1056 images belonging to 2 classes.
